In [ ]:
# OpenWeatherMap API Raw Data Analysis

This notebook fetches and displays raw weather data including precipitation values from OpenWeatherMap API.


In [ ]:
import httpx
import json
from dotenv import dotenv_values
import pandas as pd
from IPython.display import JSON

# Load environment variables
config = dotenv_values(".env")
OPENWEATHER_API_KEY = config.get("ENV_OPENWEATHER_API_KEY")
OPENWEATHER_API_BASE = "https://api.openweathermap.org/data/2.5"


In [ ]:
async def fetch_weather_data(city: str, country_code: str = ""):
    """Fetch both current weather and forecast data for analysis"""
    location = f"{city},{country_code}" if country_code else city
    
    # Parameters for API calls
    params = {
        "q": location,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"
    }
    
    async with httpx.AsyncClient() as client:
        # Fetch current weather
        weather_response = await client.get(
            f"{OPENWEATHER_API_BASE}/weather",
            params=params
        )
        weather_data = weather_response.json()
        
        # Fetch forecast data
        forecast_response = await client.get(
            f"{OPENWEATHER_API_BASE}/forecast",
            params=params
        )
        forecast_data = forecast_response.json()
        
        return weather_data, forecast_data


In [ ]:
import asyncio

# Example: Fetch data for London
weather_data, forecast_data = await fetch_weather_data("London", "UK")

# Display raw current weather data
print("Current Weather Raw Data:")
JSON(weather_data)


In [ ]:
# Display forecast data with precipitation probability
print("\nForecast Raw Data (including precipitation probability):")
JSON(forecast_data)


In [ ]:
# Create a DataFrame with forecast data for better analysis
forecast_items = forecast_data['list']
forecast_df = pd.DataFrame([
    {
        'time': item['dt_txt'],
        'temperature': item['main']['temp'],
        'precipitation_prob': item.get('pop', 0) * 100,  # Convert to percentage
        'humidity': item['main']['humidity'],
        'description': item['weather'][0]['description']
    }
    for item in forecast_items
])

print("Forecast data in tabular format:")
forecast_df


In [ ]:
# Visualize precipitation probability over time
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(forecast_df['time'], forecast_df['precipitation_prob'], marker='o')
plt.title('Precipitation Probability Over Time')
plt.xlabel('Time')
plt.ylabel('Precipitation Probability (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()
